In [1]:
!pip install keract

In [2]:
import os
from glob import glob
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn
from skimage.transform import resize
from skimage.color import gray2rgb
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import SVG
# import keract
from tensorflow import keras
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

2024-02-27 18:18:29.124588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 18:18:29.124716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 18:18:29.248557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
img_width, img_height = 150, 150 
channels = 3
batch_size = 32

In [4]:
train_path="/kaggle/input/kermany2018/OCT2017 /train"
val_path="/kaggle/input/kermany2018/OCT2017 /val"
test_path="/kaggle/input/kermany2018/OCT2017 /test"
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range= (0.73, 0.9),
    horizontal_flip= True,
    rotation_range= 10,
    width_shift_range= 0.10,
    fill_mode= 'constant',
    height_shift_range= 0.10,   
    brightness_range= (0.55, 0.9)
)
val_datagen = ImageDataGenerator(
    rescale= 1./255, 
)

train_dataset=train_datagen.flow_from_directory(  
    train_path,  
    target_size= (150, 150), 
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)
val_dataset = val_datagen.flow_from_directory(
    val_path,
    target_size= (150, 150),
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)
test_dataset = val_datagen.flow_from_directory(  
    test_path,  
    target_size= (150, 150), 
    color_mode= 'rgb',
    batch_size= 32,        
    class_mode= 'categorical',
    shuffle= False, 
)

Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [5]:
num_classes = len(train_dataset.class_indices)  
train_labels = train_dataset.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = val_dataset.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_dataset.filenames)  
nb_valid_samples = len(val_dataset.filenames)
nb_test_samples = len(test_dataset.filenames)

In [6]:
base_model=keras.models.load_model("/kaggle/input/oct-3classes/finetuned_model.h5")

In [7]:
model=Sequential()
for layer in base_model.layers[:-1]:
    model.add(layer)
model.build(input_shape=(None,150, 150, 3))
for layer in model.layers:
    layer.trainable= False
model.add(Dense(4,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 37, 37, 256)       2

In [8]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [9]:
checkpoint = ModelCheckpoint(
    'baseline_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

csvlogger = CSVLogger(
    filename= "baseline_training_csv.log",
    separator = ",",
    append = False
)

reduceLR = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [10]:
history = model.fit(
    train_dataset, 
    epochs = 10,
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data = val_dataset, 
    validation_steps = 1,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/10


I0000 00:00:1709057970.742023      94 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_loss improved from inf to 0.01648, saving model to baseline_model.h5
2608/2608 - 1072s - loss: 0.1535 - accuracy: 0.9480 - val_loss: 0.0165 - val_accuracy: 1.0000 - lr: 0.0010 - 1072s/epoch - 411ms/step
Epoch 2/10


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 0.01648
2608/2608 - 666s - loss: 0.1333 - accuracy: 0.9552 - val_loss: 0.0305 - val_accuracy: 1.0000 - lr: 0.0010 - 666s/epoch - 255ms/step
Epoch 3/10

Epoch 3: val_loss improved from 0.01648 to 0.01295, saving model to baseline_model.h5
2608/2608 - 663s - loss: 0.1307 - accuracy: 0.9561 - val_loss: 0.0129 - val_accuracy: 1.0000 - lr: 0.0010 - 663s/epoch - 254ms/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.01295
2608/2608 - 673s - loss: 0.1279 - accuracy: 0.9562 - val_loss: 0.0172 - val_accuracy: 1.0000 - lr: 0.0010 - 673s/epoch - 258ms/step
Epoch 5/10

Epoch 5: val_loss did not improve from 0.01295
2608/2608 - 670s - loss: 0.1271 - accuracy: 0.9570 - val_loss: 0.0166 - val_accuracy: 1.0000 - lr: 0.0010 - 670s/epoch - 257ms/step
Epoch 6/10

Epoch 6: val_loss improved from 0.01295 to 0.01192, saving model to baseline_model.h5
2608/2608 - 660s - loss: 0.1263 - accuracy: 0.9579 - val_loss: 0.0119 - val_accuracy: 1.0000 - lr: 0.0010 - 66

In [11]:
(eval_loss, eval_accuracy) = model.evaluate(test_dataset, batch_size = batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

31/31 [==============================] - 8s 270ms/step - loss: 0.0425 - accuracy: 0.9917
Test Loss:  0.04251205176115036
Test Accuracy:  0.9917355179786682


In [12]:
model.trainable= True
model.compile(optimizer= keras.optimizers.Adam(1e-5), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [13]:
checkpoint = ModelCheckpoint(
    'finetuned_model.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

csvlogger = CSVLogger(
    filename= "finetuned_training_csv.log",
    separator = ",",
    append = False
)

reduceLR = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [14]:
history_1 = model.fit(
    train_dataset, 
    epochs = 10,
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data = val_dataset, 
    validation_steps = 1,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/10

Epoch 1: loss improved from inf to 0.12953, saving model to finetuned_model.h5
2608/2608 - 702s - loss: 0.1295 - accuracy: 0.9570 - val_loss: 0.0025 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 702s/epoch - 269ms/step
Epoch 2/10

Epoch 2: loss improved from 0.12953 to 0.11495, saving model to finetuned_model.h5
2608/2608 - 685s - loss: 0.1150 - accuracy: 0.9615 - val_loss: 0.0060 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 685s/epoch - 263ms/step
Epoch 3/10

Epoch 3: loss improved from 0.11495 to 0.10455, saving model to finetuned_model.h5
2608/2608 - 678s - loss: 0.1046 - accuracy: 0.9652 - val_loss: 0.0118 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 678s/epoch - 260ms/step
Epoch 4/10

Epoch 4: loss improved from 0.10455 to 0.09825, saving model to finetuned_model.h5
2608/2608 - 682s - loss: 0.0983 - accuracy: 0.9670 - val_loss: 0.0150 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 682s/epoch - 262ms/step
Epoch 5/10

Epoch 5: loss improved from 0.09825 to 0.09046, saving model to fine

In [15]:
(eval_loss, eval_accuracy) = model.evaluate(test_dataset, batch_size= batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

31/31 [==============================] - 4s 116ms/step - loss: 0.0161 - accuracy: 0.9938
Test Loss:  0.01608581840991974
Test Accuracy:  0.9938016533851624


In [16]:
model.save_weights("Classes_3to2_Model.h5")